In [ ]:
!pip install pyLDAvis

In [317]:
import pandas as pd
import nltk
import re, string, random
import warnings

from collections import Counter
from spacy.matcher import Matcher 
from numpy import nan
from nltk.sentiment import SentimentIntensityAnalyzer
from statistics import mean
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier
from pprint import pprint

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


nlp = spacy.load("en_core_web_sm")

nltk.download([
     "names",
     'wordnet',
     "stopwords",
     "state_union",
     "twitter_samples",
     "movie_reviews",
     "averaged_perceptron_tagger",
     "vader_lexicon",
     "punkt",
     'omw-1.4',
     'universal_tagset', ])

warnings.filterwarnings('ignore',category=DeprecationWarning)
# nlp.max_length = 1850000

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to /root/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Pack

In [3]:
df = pd.read_csv("/content/Leverage_Collaboration_data_analysis.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304 entries, 0 to 303
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   ResponseId  304 non-null    object
 1   Q19         214 non-null    object
 2   Q51         204 non-null    object
 3   Q52         201 non-null    object
 4   Q57         188 non-null    object
 5   Q55         184 non-null    object
 6   Q58         184 non-null    object
 7   Q59         183 non-null    object
dtypes: object(8)
memory usage: 19.1+ KB


In [5]:
# QUESTION 51 ANALYSIS
# What kinds of information would be useful to you for evaluating an 
# individual participant's contributions to a collaborative group?

all_text_51 = df.Q51.str.cat(sep = ' ')
doc_51 = nlp(all_text_51)

In [6]:
words = [token.lemma_ for token in doc_51 if not token.is_stop and not token.is_punct] 
word_freq = Counter(words) 
word_freq.most_common(30)

[('group', 81),
 ('work', 75),
 ('time', 43),
 ('contribute', 43),
 ('contribution', 42),
 (' ', 40),
 ('individual', 40),
 ('project', 31),
 ('know', 27),
 ('member', 26),
 ('effort', 25),
 ('spend', 25),
 ('idea', 25),
 ('person', 23),
 ('useful', 23),
 ('information', 20),
 ('participant', 20),
 ('meeting', 19),
 ('participate', 17),
 ('complete', 16),
 ('evaluate', 16),
 ('think', 16),
 ('collaboration', 15),
 ('team', 14),
 ('peer', 14),
 ('try', 13),
 ('task', 13),
 ('discussion', 12),
 ('people', 12),
 ('help', 12)]

In [7]:
matcher = Matcher(nlp.vocab) 
pattern = [{'POS':'NOUN'}, {'POS':'NOUN'}]
pattern1 = [{'POS':'NOUN'}, {'POS':'VERB'}]
pattern4 = [{'POS':'NOUN'}, {"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]  
pattern5 = [{'POS':'NOUN'}, {'POS':'NOUN'},{"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]

matcher.add('PATTERN', [pattern]) 
matcher.add('PATTERN1', [pattern1])
matcher.add('RULE4',[pattern4])
matcher.add('RULE5',[pattern5]) 
matches = matcher(doc_51, as_spans=True) 
phrases = [] 
for span in matches:
    phrases.append(span.text.lower())

phrase_freq = Counter(phrases)
phrase_freq.most_common(15)

[('work done', 16),
 ('group members', 13),
 ('time spent', 12),
 ('group finish', 6),
 ('group meetings', 6),
 ('peer reviews', 6),
 ('group meeting', 4),
 ('work produced', 4),
 ('lines written', 4),
 ('person contribute', 4),
 ('person did', 4),
 ('group members can participate', 3),
 ('members can participate', 3),
 ('issues came', 2),
 ('time spend', 2)]

In [8]:
# QUESTION 52 ANALYSIS
# What kinds of information would be useful to you for evaluating collaboration quality at a group level?

all_text_52 = df.Q52.str.cat(sep = ' ')
doc_52 = nlp(all_text_52)

In [9]:
words = [token.lemma_ for token in doc_52 if not token.is_stop and not token.is_punct] 
word_freq = Counter(words) 
word_freq.most_common(30)

[('group', 135),
 ('work', 67),
 ('collaboration', 60),
 ('member', 52),
 (' ', 44),
 ('quality', 43),
 ('project', 26),
 ('time', 25),
 ('level', 24),
 ('people', 21),
 ('good', 21),
 ('idea', 21),
 ('useful', 21),
 ('team', 20),
 ('product', 19),
 ('individual', 19),
 ('contribute', 17),
 ('final', 15),
 ('information', 15),
 ('evaluate', 15),
 ('progress', 15),
 ('task', 14),
 ('complete', 13),
 ('overall', 13),
 ('person', 13),
 ('like', 12),
 ('think', 12),
 ('spend', 12),
 ('know', 10),
 ('goal', 10)]

In [10]:
matcher = Matcher(nlp.vocab) 
pattern = [{'POS':'NOUN'}, {'POS':'NOUN'}]
pattern1 = [{'POS':'NOUN'}, {'POS':'VERB'}]
pattern4 = [{'POS':'NOUN'}, {"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]  
pattern5 = [{'POS':'NOUN'}, {'POS':'NOUN'},{"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]

matcher.add('PATTERN', [pattern]) 
matcher.add('PATTERN1', [pattern1])
matcher.add('RULE4',[pattern4])
matcher.add('RULE5',[pattern5]) 
matches = matcher(doc_52, as_spans=True) 
phrases = [] 
for span in matches:
    phrases.append(span.text.lower())

phrase_freq = Counter(phrases)
phrase_freq.most_common(15)

[('collaboration quality', 19),
 ('group members', 14),
 ('group level', 11),
 ('time spent', 8),
 ('group works', 6),
 ('work completed', 6),
 ('work done', 6),
 ('group member', 6),
 ('group dynamic', 4),
 ('group stayed', 4),
 ('people get', 4),
 ('person contributed', 4),
 ('progress made', 4),
 ('end product', 4),
 ('team member', 3)]

In [11]:
# QUESTION 57 ANALYSIS
# What kind of collaboration-quality information would you like to see in real-time?

all_text_57 = df.Q57.str.cat(sep = ' ')
doc_57 = nlp(all_text_57)

In [12]:
words = [token.lemma_ for token in doc_57 if not token.is_stop and not token.is_punct] 
word_freq = Counter(words) 
word_freq.most_common(30)

[('group', 53),
 ('work', 50),
 ('collaboration', 43),
 ('time', 39),
 ('like', 37),
 (' ', 36),
 ('member', 32),
 ('people', 28),
 ('real', 24),
 ('quality', 19),
 ('information', 19),
 ('progress', 18),
 ('person', 17),
 ('task', 16),
 ('participate', 13),
 ('team', 13),
 ('talk', 13),
 ('discussion', 12),
 ('help', 12),
 ('contribute', 11),
 ('level', 11),
 ('idea', 11),
 ('sure', 11),
 ('project', 10),
 ('speak', 10),
 ('different', 10),
 ('job', 10),
 ('think', 10),
 ('problem', 9),
 ('collaborate', 9)]

In [13]:
matcher = Matcher(nlp.vocab) 
pattern = [{'POS':'NOUN'}, {'POS':'NOUN'}]
pattern1 = [{'POS':'NOUN'}, {'POS':'VERB'}]
pattern4 = [{'POS':'NOUN'}, {"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]  
pattern5 = [{'POS':'NOUN'}, {'POS':'NOUN'},{"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]

matcher.add('PATTERN', [pattern]) 
matcher.add('PATTERN1', [pattern1])
matches = matcher(doc_57, as_spans=True) 
phrases = [] 
for span in matches:
    phrases.append(span.text.lower())

phrase_freq = Counter(phrases)
phrase_freq.most_common(30)

[('group members', 10),
 ('quality information', 8),
 ('team members', 5),
 ('collaboration quality', 5),
 ('time collaboration', 4),
 ('group member', 4),
 ('group finish', 3),
 ('peer reviews', 3),
 ('task progress', 2),
 ('work done', 2),
 ('body language', 2),
 ('time spent', 2),
 ('problem set', 2),
 ('time include', 2),
 ('completion time', 1),
 ('group understand', 1),
 ('computer science', 1),
 ('science assignments', 1),
 ('collaboration information', 1),
 ('information sounds', 1),
 ('code written', 1),
 ('time invested', 1),
 ('people don‚äôt', 1),
 ('don‚äôt feel', 1),
 ('progress made', 1),
 ('work progress', 1),
 ('progress chart', 1),
 ('reviews judging', 1),
 ('groups works', 1),
 ('members speak', 1)]

In [14]:
# QUESTION 55 ANALYSIS
# How would you use this real-time information?

all_text_55 = df.Q55.str.cat(sep = ' ')
doc_55 = nlp(all_text_55)

In [15]:
words = [token.lemma_ for token in doc_55 if not token.is_stop and not token.is_punct] 
word_freq = Counter(words) 
word_freq.most_common(30)

[('group', 65),
 ('use', 56),
 ('collaboration', 52),
 ('information', 43),
 ('work', 42),
 ('time', 36),
 (' ', 32),
 ('member', 31),
 ('people', 26),
 ('real', 25),
 ('help', 19),
 ('participate', 17),
 ('know', 16),
 ('need', 15),
 ('adjust', 12),
 ('team', 12),
 ('task', 12),
 ('try', 12),
 ('improve', 12),
 ('progress', 12),
 ('project', 11),
 ('understand', 11),
 ('talk', 11),
 ('encourage', 11),
 ('good', 10),
 ('collaborate', 10),
 ('job', 10),
 ('quality', 10),
 ('evaluate', 10),
 ('idea', 9)]

In [16]:

matcher = Matcher(nlp.vocab) 
pattern = [{'POS':'NOUN'}, {'POS':'NOUN'}]
pattern1 = [{'POS':'NOUN'}, {'POS':'VERB'}]
pattern4 = [{'POS':'NOUN'}, {"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]  
pattern5 = [{'POS':'NOUN'}, {'POS':'NOUN'},{"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]

matcher.add('PATTERN', [pattern]) 
matcher.add('PATTERN1', [pattern1])
matcher.add('RULE4',[pattern4])
matcher.add('RULE5',[pattern5]) 
matches = matcher(doc_55, as_spans=True) 
phrases = [] 
for span in matches:
    phrases.append(span.text.lower())

phrase_freq = Counter(phrases)
phrase_freq.most_common(15)

[('time information', 19),
 ('group members', 12),
 ('group finish', 6),
 ('people engaged', 4),
 ('people have', 4),
 ('collaboration quality', 4),
 ('points made', 4),
 ('group members can participate', 3),
 ('members can participate', 3),
 ('tasks tie', 2),
 ('individuals talk', 2),
 ('deadlines based', 2),
 ('group dynamic', 2),
 ('people disengaged', 2),
 ('communication makes', 2)]

In [17]:
# QUESTION 58 ANALYSIS
# What kind of collaboration-quality information would you like to see after the collaboration?

all_text_58 = df.Q58.str.cat(sep = ' ')
doc_58 = nlp(all_text_58)

In [18]:
words = [token.lemma_ for token in doc_58 if not token.is_stop and not token.is_punct] 
word_freq = Counter(words) 
word_freq.most_common(30)

[('collaboration', 66),
 ('group', 53),
 ('like', 45),
 ('member', 43),
 ('work', 37),
 (' ', 33),
 ('project', 21),
 ('feel', 20),
 ('information', 19),
 ('people', 17),
 ('think', 17),
 ('time', 16),
 ('quality', 16),
 ('individual', 14),
 ('contribute', 14),
 ('result', 13),
 ('person', 13),
 ('team', 11),
 ('feedback', 11),
 ('go', 11),
 ('overall', 10),
 ('final', 10),
 ('want', 10),
 ('improve', 9),
 ('good', 9),
 ('peer', 9),
 ('maybe', 9),
 ('job', 9),
 ('product', 9),
 ('contribution', 9)]

In [19]:
matcher = Matcher(nlp.vocab) 
pattern = [{'POS':'NOUN'}, {'POS':'NOUN'}]
pattern1 = [{'POS':'NOUN'}, {'POS':'VERB'}]
pattern4 = [{'POS':'NOUN'}, {"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]  
pattern5 = [{'POS':'NOUN'}, {'POS':'NOUN'},{"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]

matcher.add('PATTERN', [pattern]) 
matcher.add('PATTERN1', [pattern1])
#matcher.add('RULE4',[pattern4])
#matcher.add('RULE5',[pattern5]) 
matches = matcher(doc_58, as_spans=True) 
phrases = [] 
for span in matches:
    phrases.append(span.text.lower())

phrase_freq = Counter(phrases)
phrase_freq.most_common(30)

[('group members', 13),
 ('collaboration went', 4),
 ('team members', 4),
 ('quality information', 3),
 ('group finish', 3),
 ('work done', 3),
 ('peer reviews', 3),
 ('collaboration quality', 2),
 ('member thought', 2),
 ('people felt', 2),
 ('team member', 2),
 ('member contributed', 2),
 ('peer evaluations', 2),
 ('time taken', 1),
 ('team satisfaction', 1),
 ('pain points', 1),
 ('peer ratings', 1),
 ('team felt', 1),
 ('member felt', 1),
 ('group survey', 1),
 ('work contributed', 1),
 ('quality checks', 1),
 ('group did', 1),
 ('satisfaction level', 1),
 ('group dynamic', 1),
 ('grade aspect', 1),
 ('quality indicator', 1),
 ('people write', 1),
 ('group discussed', 1),
 ('person feel', 1)]

In [325]:
# QUESTION 59 ANALYSIS
# What kind of collaboration-quality information would you like to see after the collaboration?

all_text_59 = df.Q59.str.cat(sep = ' ')
doc_59 = nlp(all_text_59)

words = [token.lemma_ for token in doc_59 if not token.is_stop and not token.is_punct] 
word_freq = Counter(words) 
word_freq.most_common(30)

matcher = Matcher(nlp.vocab) 
pattern = [{'POS':'NOUN'}, {'POS':'NOUN'}]
pattern1 = [{'POS':'NOUN'}, {'POS':'VERB'}]
pattern4 = [{'POS':'NOUN'}, {"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]  
pattern5 = [{'POS':'NOUN'}, {'POS':'NOUN'},{"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]

matcher.add('PATTERN', [pattern]) 
matcher.add('PATTERN1', [pattern1])
#matcher.add('RULE4',[pattern4])
#matcher.add('RULE5',[pattern5]) 
matches = matcher(doc_59, as_spans=True) 
phrases = [] 
for span in matches:
    phrases.append(span.text.lower())

phrase_freq = Counter(phrases)
phrase_freq.most_common(30)

[('group members', 7),
 ('group work', 4),
 ('group member', 4),
 ('group dynamics', 4),
 ('group finish', 3),
 ('collaboration quality', 3),
 ('team members', 2),
 ('end product', 2),
 ('group level', 2),
 ('group using', 1),
 ('pain points', 1),
 ('work distribution', 1),
 ('person thought', 1),
 ('work moves', 1),
 ('group dynamic', 1),
 ('personality types', 1),
 ('team atmosphere', 1),
 ('project setting', 1),
 ('collaboration approach', 1),
 ('work sessions', 1),
 ('team improve', 1),
 ('student group', 1),
 ('group assignment', 1),
 ('group knows', 1),
 ('time contributing', 1),
 ('peers contribution', 1),
 ('team project', 1),
 ('collaboration sessions', 1),
 ('peer evaluations', 1),
 ('project evaluation', 1)]

In [ ]:
# pattern2 = [{'POS':'VERB'}, {"POS": "DET", "OP" : "*"},{'POS':'NOUN'}]
# pattern3 = [{'POS':'VERB'}, {"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'NOUN'}]  
# matcher.add('VRB_PHRASE', [pattern2]) 
# matcher.add('VRB_PHRASE', [pattern3]) 

#pattern4 = [{'POS':'NOUN'}, {"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]  
#pattern5 = [{'POS':'NOUN'}, {'POS':'NOUN'},{"POS": "AUX", "OP" : "*"},{"POS": "ADP", "OP" : "*"},{"POS": "DET", "OP" : "*"},{'POS':'VERB'}]
#matcher.add('RULE4',[pattern4])
#matcher.add('RULE5',[pattern5]) 

In [20]:
sia = SentimentIntensityAnalyzer()
positive_review_ids = nltk.corpus.movie_reviews.fileids(categories=["pos"])
negative_review_ids = nltk.corpus.movie_reviews.fileids(categories=["neg"])
all_review_ids = positive_review_ids + negative_review_ids

In [21]:
def is_positive(review_id: str) -> bool:
    """True if the average of all sentence compound scores is positive."""
    text = nltk.corpus.movie_reviews.raw(review_id)
    #print(text)
    scores = [
        sia.polarity_scores(sentence)["compound"]
        for sentence in nltk.sent_tokenize(text)
    ]
    return mean(scores) > 0


In [22]:
correct = 0
for review_id in all_review_ids:
     if is_positive(review_id):
        if review_id in positive_review_ids:
             correct += 1
     else:
         if review_id in negative_review_ids:
             correct += 1

print(F"{correct / len(all_review_ids):.2%} correct")

64.05% correct


In [23]:
unwanted = nltk.corpus.stopwords.words("english")

unwanted.extend([w.lower() for w in nltk.corpus.names.words()])


def skip_unwanted(pos_tuple):

    word, tag = pos_tuple

    if not word.isalpha() or word in unwanted:

        return False

    if tag.startswith("NN"):

        return False

    return True


positive_words = [word for word, tag in filter(

    skip_unwanted,

    nltk.pos_tag(nltk.corpus.movie_reviews.words(categories=["pos"]))

)]

negative_words = [word for word, tag in filter(

    skip_unwanted,

    nltk.pos_tag(nltk.corpus.movie_reviews.words(categories=["neg"]))

)]


In [24]:
positive_fd = nltk.FreqDist(positive_words)
negative_fd = nltk.FreqDist(negative_words)

common_set = set(positive_fd).intersection(negative_fd)

for word in common_set:
    del positive_fd[word]
    del negative_fd[word]

top_100_positive = {word for word, count in positive_fd.most_common(100)}
top_100_negative = {word for word, count in negative_fd.most_common(100)}


In [ ]:
# USING MOVIE REVIEW SAMPLES TO AS TRAINIG DATA

In [25]:
def extract_features(text):
    features = dict()
    wordcount = 0
    compound_scores = list()
    positive_scores = list()

    for sentence in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sentence):
            if word.lower() in positive_fd:
                wordcount += 1
        compound_scores.append(sia.polarity_scores(sentence)["compound"])
        positive_scores.append(sia.polarity_scores(sentence)["pos"])

    # Adding 1 to the final compound score to always have positive numbers
    # since some classifiers  don't work with negative numbers.
    features["mean_compound"] = mean(compound_scores) + 1
    features["mean_positive"] = mean(positive_scores)
    features["wordcount"] = wordcount

    return features


In [26]:
positives = 0
negatives = 0
for resp in df.Q51:
  if resp is not nan:
    features = extract_features(resp)
    if features["mean_positive"] > 0:
      positives+=1
    else:
       negatives+=1 
    #print(features)
print("Sentiments --- Positives: ", positives, " Negatives: ",negatives)    


Sentiments --- Positives:  113  Negatives:  91


In [ ]:
# USING TWITTER SAMPLES TO AS TRAINIG DATA 

In [27]:

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

if __name__ == "__main__":

    positive_tweets = twitter_samples.strings('positive_tweets.json')
    negative_tweets = twitter_samples.strings('negative_tweets.json')
    text = twitter_samples.strings('tweets.20150430-223406.json')
    tweet_tokens = twitter_samples.tokenized('positive_tweets.json')[0]

    stop_words = stopwords.words('english')

    positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
    negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

    positive_cleaned_tokens_list = []
    negative_cleaned_tokens_list = []

    for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    all_pos_words = get_all_words(positive_cleaned_tokens_list)

    freq_dist_pos = FreqDist(all_pos_words)
    print(freq_dist_pos.most_common(10))

    positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
    negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

    positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

    negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

    dataset = positive_dataset + negative_dataset

    random.shuffle(dataset)

    train_data = dataset[:7000]
    test_data = dataset[7000:]

    classifier = NaiveBayesClassifier.train(train_data)

    #print("Accuracy is:", classify.accuracy(classifier, test_data))

    #print(classifier.show_most_informative_features(10))

    

<>:6: DeprecationWarning: invalid escape sequence \(
<>:6: DeprecationWarning: invalid escape sequence \(
<ipython-input-27-4c9149160059>:6: DeprecationWarning: invalid escape sequence \(
  token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\


[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


In [28]:
positives = 0
negatives = 0
for resp in df.Q51:
  if resp is not nan:
    custom_tokens = remove_noise(word_tokenize(resp))
    cat = classifier.classify(dict([token, True] for token in custom_tokens))
    if cat == 'Positive':
      positives+=1
    else:
       negatives+=1 
    #print(features)
print("Sentiments --- Positives: ", positives, " Negatives: ",negatives)

Sentiments --- Positives:  148  Negatives:  56


In [ ]:
#SENTIMENT ANALYSIS FOR Q51,Q55,Q57.Q58.Q59

In [29]:
res = [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]
print("   sno    ","      Q51    ","            Q52        ","        Q55        ","        Q57        ","       Q58       ","        Q59        ","pos:neg   ")
for i in range(len(df.Q51)):
  positives = 0
  negatives = 0
  if df.Q51[i] is not nan and df.Q52[i] is not nan and df.Q55[i] is not nan and df.Q57[i] is not nan and df.Q58[i] is not nan and df.Q59[i] is not nan:
      custom_tokens = remove_noise(word_tokenize(df.Q51[i]))
      Q51 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q51 == 'Positive':
        res[0][0]+=1
        positives+=1
      else:
        res[0][1]+=1
        negatives+=1
    
      custom_tokens = remove_noise(word_tokenize(df.Q52[i]))
      Q52 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q52 == 'Positive':
        res[1][0]+=1
        positives+=1
      else:
        res[1][1]+=1
        negatives+=1
    

      custom_tokens = remove_noise(word_tokenize(df.Q55[i]))
      Q55 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q55 == 'Positive':
        res[2][0]+=1
        positives+=1
      else:
        res[2][1]+=1
        negatives+=1
    
      custom_tokens = remove_noise(word_tokenize(df.Q57[i]))
      Q57 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q57 == 'Positive':
        res[3][0]+=1
        positives+=1
      else:
        res[3][1]+=1
        negatives+=1
    

      custom_tokens = remove_noise(word_tokenize(df.Q58[i]))
      Q58 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q58 == 'Positive':
        res[4][0]+=1
        positives+=1
      else:
        res[4][1]+=1
        negatives+=1
      

      custom_tokens = remove_noise(word_tokenize(df.Q59[i]))
      Q59 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q59 == 'Positive':
        res[5][0]+=1
        positives+=1
      else:
        res[5][1]+=1
        negatives+=1
    
      print("   ",i+1,"    ","   ",Q51,"    ","    ",Q52,"    ","    ",Q55,"    ","    ",Q57,"    ","    ",Q58,"    ","    ",Q59,"      ",positives,":",negatives)                              
    

   sno           Q51                 Q52                 Q55                 Q57                Q58                Q59         pos:neg   
    11          Positive           Positive           Positive           Negative           Positive           Positive        5 : 1
    12          Negative           Positive           Positive           Positive           Positive           Positive        5 : 1
    13          Positive           Positive           Negative           Positive           Negative           Positive        4 : 2
    14          Negative           Positive           Negative           Negative           Negative           Negative        1 : 5
    15          Positive           Positive           Positive           Positive           Negative           Positive        5 : 1
    16          Negative           Positive           Negative           Positive           Positive           Positive        4 : 2
    17          Negative           Negative           Negative  

In [30]:
print("   pos/neg    ","      Q51    ","            Q52        ","        Q55        ","        Q57        ","       Q58       ","        Q59        ")
print("   Positives    ","   ",res[0][0],"    ","          ",res[1][0],"        ","      ",res[2][0],"       ","       ",res[3][0],"        ","     ",res[4][0],"        ","     ",res[5][0],"    ")
print("   Negatives    ","   ",res[0][1],"      ","          ",res[1][1],"        ","      ",res[2][1],"       ","         ",res[3][1],"        ","     ",res[4][1],"        ","       ",res[5][1],"    ")

   pos/neg           Q51                 Q52                 Q55                 Q57                Q58                Q59        
   Positives         128                 120                 109                 110                110                127     
   Negatives         52                   60                 71                   70                70                  53     


In [ ]:
#SENTIMENT ANALYSIS FOR Q19, Q51,Q55,Q57.Q58.Q59

In [31]:
res = [[0,0],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]]
print("   sno    ","      Q19    ","          Q51      ","           Q52        ","        Q55        ","      Q57        ","       Q58       ","        Q59        ","pos:neg   ")
for i in range(len(df.Q51)):
  positives = 0
  negatives = 0
  if df.Q19[i] is not nan and df.Q51[i] is not nan and df.Q52[i] is not nan and df.Q55[i] is not nan and df.Q57[i] is not nan and df.Q58[i] is not nan and df.Q59[i] is not nan:
      custom_tokens = remove_noise(word_tokenize(df.Q19[i]))
      Q19 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q19 == 'Positive':
        res[0][0]+=1
        positives+=1
      else:
        res[0][1]+=1
        negatives+=1

      custom_tokens = remove_noise(word_tokenize(df.Q51[i]))
      Q51 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q51 == 'Positive':
        res[1][0]+=1
        positives+=1
      else:
        res[1][1]+=1
        negatives+=1
    
      custom_tokens = remove_noise(word_tokenize(df.Q52[i]))
      Q52 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q52 == 'Positive':
        res[2][0]+=1
        positives+=1
      else:
        res[2][1]+=1
        negatives+=1
    

      custom_tokens = remove_noise(word_tokenize(df.Q55[i]))
      Q55 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q55 == 'Positive':
        res[3][0]+=1
        positives+=1
      else:
        res[3][1]+=1
        negatives+=1
    
      custom_tokens = remove_noise(word_tokenize(df.Q57[i]))
      Q57 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q57 == 'Positive':
        res[4][0]+=1
        positives+=1
      else:
        res[4][1]+=1
        negatives+=1
    

      custom_tokens = remove_noise(word_tokenize(df.Q58[i]))
      Q58 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q58 == 'Positive':
        res[5][0]+=1
        positives+=1
      else:
        res[5][1]+=1
        negatives+=1
      

      custom_tokens = remove_noise(word_tokenize(df.Q59[i]))
      Q59 = classifier.classify(dict([token, True] for token in custom_tokens))
      if Q59 == 'Positive':
        res[6][0]+=1
        positives+=1
      else:
        res[6][1]+=1
        negatives+=1
    
      print("   ",i+1,"    ","   ",Q19,"    ","   ",Q51,"    ","    ",Q52,"    ","    ",Q55,"    ","    ",Q57,"    ","    ",Q58,"    ","    ",Q59,"      ",positives,":",negatives)  

   sno           Q19               Q51                  Q52                 Q55               Q57                Q58                Q59         pos:neg   
    11          Positive          Positive           Positive           Positive           Negative           Positive           Positive        6 : 1
    12          Positive          Negative           Positive           Positive           Positive           Positive           Positive        6 : 1
    13          Positive          Positive           Positive           Negative           Positive           Negative           Positive        5 : 2
    14          Negative          Negative           Positive           Negative           Negative           Negative           Negative        1 : 6
    15          Negative          Positive           Positive           Positive           Positive           Negative           Positive        5 : 2
    16          Positive          Negative           Positive           Negative          

In [32]:
print("   pos/neg    ","      Q19    ","      Q51    ","            Q52        ","        Q55        ","        Q57        ","       Q58       ","        Q59        ")
print("   Positives    ","   ",res[0][0],"    ","   ",res[1][0],"    ","          ",res[2][0],"        ","      ",res[3][0],"       ","       ",res[4][0],"        ","     ",res[5][0],"        ","     ",res[6][0],"    ")
print("   Negatives    ","   ",res[0][0],"    ","   ",res[1][1],"      ","          ",res[2][1],"        ","      ",res[3][1],"       ","         ",res[4][1],"        ","     ",res[5][1],"        ","       ",res[6][1],"    ")

   pos/neg           Q19           Q51                 Q52                 Q55                 Q57                Q58                Q59        
   Positives         99          121                 113                 104                 104                103                119     
   Negatives         99          50                   58                 67                   67                68                  52     


In [33]:
text = nltk.Text(nltk.word_tokenize(all_text_58))
text.concordance("peer", lines=5)

Displaying 5 of 8 matches:
collaboration Time spent , anonymous peer ratings on collaboration Maybe some 
 would like to see feedback report , peer evaluations and project evaluation a
of a specific idea or path to take . peer reviews Each person providing an ano
 everyone said . I would like to see peer evaluations to see how well the grou
t of the work Similar to real-time , peer review but maybe with the addition o


In [34]:
concordance_list = text.concordance_list("group", lines=5)
for entry in concordance_list:
    print(entry.line)

the collaboration we could include a group survey just to see what everyone tho
 will tell them that only all of our group members can participate the job acti
help with each other , can the whole group finish the job.I will try to communi
 will tell them that only all of our group members can participate the job acti
help with each other , can the whole group finish the job.I will try to communi


In [35]:
token = nltk.word_tokenize(all_text_58)
stopwords = nltk.corpus.stopwords.words("english")
words_58 = [w for w in token if w.lower() not in stopwords and w.isalpha()]
text_1 = nltk.Text(words_58)
fd = text_1.vocab()
fd.tabulate(20)

collaboration         would         group           see          like          work       members          well       project   information        people          felt        member          much       Nothing      everyone        person         think          time       quality 
           58            56            51            46            45            29            28            22            19            18            16            16            15            14            14            13            13            12            11            11 


In [36]:
finder = nltk.collocations.BigramCollocationFinder.from_words(words_58)
finder.ngram_fd.most_common(10)

[(('like', 'see'), 35),
 (('would', 'like'), 33),
 (('group', 'members'), 13),
 (('Nothing', 'Nothing'), 13),
 (('participate', 'job'), 6),
 (('final', 'product'), 6),
 (('collaboration', 'would'), 6),
 (('felt', 'collaboration'), 5),
 (('want', 'see'), 5),
 (('information', 'would'), 4)]

In [ ]:
#TRYING OUT TOPIC MODELING

stopwords = nltk.corpus.stopwords.words("english")

In [315]:
#Tokenize words and Clean-up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    #stop_words.extend(['much','find','past','see','want','way','also','else','get','put','often','kind','one'])
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out



In [319]:
# TOPIC MODELING FOR Q51
#Remove emails and newline characters

#data = TM_df.content.values.tolist()
data = nltk.sent_tokenize(all_text_51)

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

#pprint(data[:2])

data_words = list(sent_to_words(data))

#print(data_words[:2])

#Creating Bigram and Trigram Models

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=5) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=5)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[100]]])

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Trigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

##print(data_lemmatized[100])


#Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(len(corpus[100]))

#Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(len(corpus[100]))

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           #chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)



# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds='mmds')
vis


/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.259613 -0.088528       1        1  12.131912
5     -0.263260  0.071994       2        1   8.670218
13    -0.163223 -0.248595       3        1   7.519214
9      0.037907 -0.256614       4        1   7.094967
3     -0.069527  0.241130       5        1   6.926423
1     -0.172486  0.008397       6        1   6.414286
11    -0.162859  0.170067       7        1   6.389868
8     -0.098341 -0.070378       8        1   5.727278
19    -0.069161 -0.143655       9        1   5.622439
16     0.041731 -0.150822      10        1   4.044136
14     0.231445 -0.139637      11        1   3.564541
10     0.110075  0.231847      12        1   3.439654
15     0.018215  0.034746      13        1   3.412241
7      0.248899  0.025538      14        1   3.375304
2     -0.061054  0.109990      15        1   3.267608
18     0.041079  0.147437      16        1   2.915820
0      0.151214 -0.125331      17        1   2.647845
17     0.128162 -0.017427      18        1   2.612191
12     0.184248  0.121938      19        1   2.591280
6      0.126549  0.077904      20        1   1.632775, topic_info=          Term       Freq      Total Category  logprob  loglift
25        work  40.000000  40.000000  Default  30.0000  30.0000
18       group  42.000000  42.000000  Default  29.0000  29.0000
1       effort  14.000000  14.000000  Default  28.0000  28.0000
66      useful  17.000000  17.000000  Default  27.0000  27.0000
75         job   7.000000   7.000000  Default  26.0000  26.0000
..         ...        ...        ...      ...      ...      ...
9         goal   0.023732   2.681727  Topic20  -6.7696  -0.6125
10        make   0.023732   2.808315  Topic20  -6.7696  -0.6586
11     project   0.023732  24.296626  Topic20  -6.7696  -2.8164
12       skill   0.023732   1.297521  Topic20  -6.7696   0.1135
13  understand   0.023732   4.127732  Topic20  -6.7696  -1.0438

[878 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
479      14  0.770701      ability
141       7  0.498282         able
141      12  0.249141         able
298       7  0.926642   accomplish
501       1  0.665010  accordingly
...     ...       ...          ...
289       7  0.200212      working
289       8  0.200212      working
289      19  0.200212      working
523       3  0.693457        worse
331      12  0.762859        write

[836 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 6, 14, 10, 4, 2, 12, 9, 20, 17, 15, 11, 16, 8, 3, 19, 1, 18, 13, 7])

In [320]:
# TOPIC MODELING FOR Q52
#Remove emails and newline characters

#data = TM_df.content.values.tolist()
data = nltk.sent_tokenize(all_text_52)

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

#pprint(data[:2])

data_words = list(sent_to_words(data))

#print(data_words[:2])

#Creating Bigram and Trigram Models

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=5) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=5)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[100]]])

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Trigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

##print(data_lemmatized[100])


#Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(len(corpus[100]))

#Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(len(corpus[100]))

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           #chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)



# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds='mmds')
vis

/usr/local/lib/python3.8/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.018253  0.240057       1        1  13.184672
11     0.052936 -0.235739       2        1  11.463039
17     0.154640  0.188402       3        1   7.466407
13    -0.075316 -0.206003       4        1   6.662379
9     -0.070276  0.187118       5        1   5.506041
10    -0.086982  0.034392       6        1   5.240511
2      0.231471 -0.012097       7        1   5.129562
1     -0.223714 -0.099150       8        1   5.064051
8      0.032578  0.144635       9        1   4.732508
6     -0.144869  0.098402      10        1   4.649909
3     -0.145354 -0.031133      11        1   4.052515
5      0.130126 -0.157343      12        1   3.999972
15    -0.238723  0.100858      13        1   3.717424
12     0.178632 -0.093631      14        1   3.465259
14     0.033494  0.049472      15        1   3.219962
7      0.154936  0.096588      16        1   2.977583
19     0.129202 -0.015732      17        1   2.660468
4     -0.000571 -0.115069      18        1   2.559889
18    -0.127537 -0.121098      19        1   2.224488
16    -0.002925 -0.052928      20        1   2.023363, topic_info=              Term       Freq      Total Category  logprob  loglift
32            well  20.000000  20.000000  Default  30.0000  30.0000
436       everyone  16.000000  16.000000  Default  29.0000  29.0000
1          quality  19.000000  19.000000  Default  28.0000  28.0000
94            time  17.000000  17.000000  Default  27.0000  27.0000
109        overall  10.000000  10.000000  Default  26.0000  26.0000
..             ...        ...        ...      ...      ...      ...
6    communication   0.025473   6.541519  Topic20  -6.9367  -1.6479
10            look   0.025473   4.214070  Topic20  -6.9367  -1.2081
11          member   0.025473  10.163500  Topic20  -6.9367  -2.0885
13         project   0.025473  18.104385  Topic20  -6.9367  -2.6659
14      seamlessly   0.025473   2.111984  Topic20  -6.9367  -0.5173

[873 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
131       4  0.303731   ability
131      11  0.303731   ability
131      15  0.303731   ability
152       4  0.140131      able
152       6  0.140131      able
...     ...       ...       ...
394      17  0.196854   working
394      18  0.196854   working
394      20  0.196854   working
487       3  0.708360  workload
189       3  0.927895     write

[866 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 12, 18, 14, 10, 11, 3, 2, 9, 7, 4, 6, 16, 13, 15, 8, 20, 5, 19, 17])

In [321]:
# TOPIC MODELING FOR Q55
#Remove emails and newline characters

#data = TM_df.content.values.tolist()
data = nltk.sent_tokenize(all_text_55)

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

#pprint(data[:2])

data_words = list(sent_to_words(data))

#print(data_words[:2])

#Creating Bigram and Trigram Models

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=5) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=5)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[100]]])

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Trigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

##print(data_lemmatized[100])


#Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(len(corpus[100]))

#Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(len(corpus[100]))

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           #chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)



# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds='mmds')
vis

/usr/local/lib/python3.8/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.256535 -0.096324       1        1  16.447131
15    -0.206947  0.106224       2        1   9.872794
14    -0.165103 -0.099633       3        1   8.687336
3     -0.011438 -0.216213       4        1   6.254122
2      0.135029  0.179776       5        1   5.435523
12    -0.081817 -0.057655       6        1   5.115169
9     -0.105457  0.154178       7        1   4.944322
4     -0.081560 -0.194168       8        1   4.833816
13     0.114921 -0.194478       9        1   4.700616
1      0.235088 -0.086337      10        1   4.610610
10     0.076606 -0.146204      11        1   4.459664
18     0.168196  0.101427      12        1   4.251685
8     -0.176970  0.046789      13        1   4.020530
19     0.016778  0.171961      14        1   3.774557
17    -0.049885  0.053960      15        1   3.121037
11     0.008804  0.230736      16        1   2.655108
5      0.177042  0.016106      17        1   2.393754
16     0.033435 -0.027389      18        1   2.030353
7      0.107210  0.003878      19        1   1.739112
6      0.062602  0.053368      20        1   0.652761, topic_info=              Term       Freq      Total Category  logprob  loglift
40           group  36.000000  36.000000  Default   30.000  30.0000
72   collaboration  36.000000  36.000000  Default   29.000  29.0000
235      would_use  30.000000  30.000000  Default   28.000  28.0000
60          people  19.000000  19.000000  Default   27.000  27.0000
56            work  24.000000  24.000000  Default   26.000  26.0000
..             ...        ...        ...      ...      ...      ...
19         clearly   0.012623   1.327986  Topic20   -6.452   0.3758
20         connect   0.012623   1.327986  Topic20   -6.452   0.3758
22             end   0.012623   4.250779  Topic20   -6.452  -0.7876
23          ensure   0.012623   5.981155  Topic20   -6.452  -1.1292
24     interaction   0.012623   1.212705  Topic20   -6.452   0.4666

[895 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
32        2  0.198125       able
32        3  0.396250       able
32        7  0.198125       able
32        8  0.198125       able
32       14  0.198125       able
...     ...       ...        ...
235      12  0.032318  would_use
235      13  0.032318  would_use
235      14  0.032318  would_use
235      15  0.032318  would_use
235      18  0.032318  would_use

[854 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 16, 15, 4, 3, 13, 10, 5, 14, 2, 11, 19, 9, 20, 18, 12, 6, 17, 8, 7])

In [322]:
# TOPIC MODELING FOR Q57
#Remove emails and newline characters

#data = TM_df.content.values.tolist()
data = nltk.sent_tokenize(all_text_57)

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

#pprint(data[:2])

data_words = list(sent_to_words(data))

#print(data_words[:2])

#Creating Bigram and Trigram Models

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=5) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=5)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[100]]])

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Trigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

##print(data_lemmatized[100])


#Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(len(corpus[100]))

#Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(len(corpus[100]))

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           #chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)



# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds='mmds')
vis

/usr/local/lib/python3.8/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.282387 -0.116836       1        1  11.906623
8      0.218838  0.127763       2        1  11.897493
1      0.126393 -0.128966       3        1   7.668878
12    -0.219947 -0.029664       4        1   7.536652
18    -0.025829 -0.225408       5        1   6.442790
16    -0.099344 -0.160362       6        1   6.057458
19    -0.202005 -0.132651       7        1   5.654162
7      0.021945  0.118857       8        1   5.258878
17     0.009309  0.228537       9        1   4.805480
13     0.066869 -0.211254      10        1   4.567886
5      0.184782  0.008233      11        1   3.806332
15    -0.071748  0.185132      12        1   3.768344
11     0.105604  0.172091      13        1   3.660077
14    -0.124888  0.116942      14        1   2.949547
9      0.081206  0.006362      15        1   2.907331
2     -0.190280  0.084521      16        1   2.810468
10    -0.059736 -0.034265      17        1   2.758084
4     -0.112374  0.011703      18        1   2.719077
6      0.020884 -0.053160      19        1   1.543181
3     -0.012065  0.032426      20        1   1.281257, topic_info=                     Term       Freq      Total Category  logprob  loglift
13               progress  14.000000  14.000000  Default  30.0000  30.0000
43          collaboration  20.000000  20.000000  Default  29.0000  29.0000
18  collaboration_quality   8.000000   8.000000  Default  28.0000  28.0000
6                everyone  12.000000  12.000000  Default  27.0000  27.0000
21              real_time  15.000000  15.000000  Default  26.0000  26.0000
..                    ...        ...        ...      ...      ...      ...
15                   task   0.018125   8.128655  Topic20  -6.5876  -1.7485
16                   time   0.018125  13.050052  Topic20  -6.5876  -2.2219
17             understand   0.018125   4.109197  Topic20  -6.5876  -1.0664
18  collaboration_quality   0.018125   8.729122  Topic20  -6.5876  -1.8198
19            information   0.018125  13.829618  Topic20  -6.5876  -2.2800

[884 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
215       4  0.331473          able
215       6  0.331473          able
215      18  0.331473          able
416       7  0.752835    accomplish
244      11  0.812973  accomplished
...     ...       ...           ...
57       11  0.099696       working
57       12  0.099696       working
346       3  0.481741       writing
346       6  0.481741       writing
141       2  0.697928           yet

[727 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 9, 2, 13, 19, 17, 20, 8, 18, 14, 6, 16, 12, 15, 10, 3, 11, 5, 7, 4])

In [323]:
# TOPIC MODELING FOR Q58
#Remove emails and newline characters

#data = TM_df.content.values.tolist()
data = nltk.sent_tokenize(all_text_58)

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

#pprint(data[:2])

data_words = list(sent_to_words(data))

#print(data_words[:2])

#Creating Bigram and Trigram Models

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=5) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=5)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[100]]])

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Trigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

##print(data_lemmatized[100])


#Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(len(corpus[100]))

#Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(len(corpus[100]))

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           #chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)



# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds='mmds')
vis

/usr/local/lib/python3.8/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
15    -0.199106 -0.063866       1        1  10.523917
4     -0.312897  0.000764       2        1  10.467557
18    -0.031083 -0.202021       3        1  10.012731
14    -0.147013 -0.205169       4        1   9.360820
9     -0.051942  0.256313       5        1   7.757524
16    -0.166006  0.064780       6        1   7.623311
11    -0.140549  0.159695       7        1   5.005915
12    -0.118897 -0.114338       8        1   4.125956
2      0.045723 -0.210819       9        1   4.064091
8      0.175160 -0.149492      10        1   3.932911
19     0.230686 -0.037276      11        1   3.883792
3      0.211635  0.102642      12        1   3.170681
1     -0.061286  0.097244      13        1   3.110539
7      0.115146  0.176674      14        1   3.076540
10     0.042380  0.188545      15        1   2.995371
0     -0.028441 -0.065979      16        1   2.940962
13     0.109984 -0.084288      17        1   2.692385
6      0.163733  0.048426      18        1   2.484545
17     0.090054  0.000920      19        1   1.459146
5      0.072720  0.037246      20        1   1.311305, topic_info=                Term       Freq      Total Category  logprob  loglift
21     collaboration  47.000000  47.000000  Default  30.0000  30.0000
47              work  21.000000  21.000000  Default  29.0000  29.0000
216  nothing_nothing   5.000000   5.000000  Default  28.0000  28.0000
4               time  11.000000  11.000000  Default  27.0000  27.0000
40           quality  12.000000  12.000000  Default  26.0000  26.0000
..               ...        ...        ...      ...      ...      ...
17           project   0.020895  15.361231  Topic20  -6.4135  -2.2659
20        understand   0.020895   5.134933  Topic20  -6.4135  -1.1702
21     collaboration   0.020895  47.341649  Topic20  -6.4135  -3.3915
22      collaborator   0.020895   2.006034  Topic20  -6.4135  -0.2303
23              fact   0.020895   1.464423  Topic20  -6.4135   0.0844

[861 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
367       4  0.689603   ability
5         1  0.446715      able
5         3  0.223357      able
5        14  0.223357      able
278       8  0.752922  abstract
...     ...       ...       ...
134       3  0.495470     write
134      18  0.495470     write
344       4  0.689603     wrong
131       3  0.682863       yet
403       7  0.739308     yield

[642 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[16, 5, 19, 15, 10, 17, 12, 13, 3, 9, 20, 4, 2, 8, 11, 1, 14, 7, 18, 6])

In [326]:
# TOPIC MODELING FOR Q59
#Remove emails and newline characters

#data = TM_df.content.values.tolist()
data = nltk.sent_tokenize(all_text_59)

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

#pprint(data[:2])

data_words = list(sent_to_words(data))

#print(data_words[:2])

#Creating Bigram and Trigram Models

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=5) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=5)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[100]]])

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Trigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

##print(data_lemmatized[100])


#Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(len(corpus[100]))

#Create the Dictionary and Corpus needed for Topic Modeling
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_words_bigrams

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(len(corpus[100]))

# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           #chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)



# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word,mds='mmds')
vis

/usr/local/lib/python3.8/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
11    -0.108128  0.177853       1        1  9.969143
13    -0.001006 -0.241901       2        1  9.362480
10     0.041730  0.253515       3        1  8.640344
0     -0.219363 -0.194603       4        1  8.254026
14    -0.170453  0.130393       5        1  6.984693
1      0.160491  0.182276       6        1  6.405987
4      0.212637  0.057195       7        1  5.987333
12    -0.217062  0.033775       8        1  5.844008
16     0.007601  0.076212       9        1  5.517187
18    -0.006869  0.184617      10        1  5.299114
17    -0.167683 -0.093907      11        1  5.092137
8     -0.064342 -0.019645      12        1  3.819949
19    -0.118537  0.006217      13        1  3.785776
6      0.216062 -0.063188      14        1  3.013123
9      0.143326 -0.164680      15        1  2.855897
7      0.119143  0.066143      16        1  2.749900
3      0.083567 -0.030431      17        1  2.095520
5      0.056296 -0.153744      18        1  1.840554
2     -0.015464 -0.123354      19        1  1.415821
15     0.048053 -0.082743      20        1  1.067011, topic_info=         Term       Freq      Total Category  logprob  loglift
14      group  32.000000  32.000000  Default  30.0000  30.0000
59     future  17.000000  17.000000  Default  29.0000  29.0000
15    improve  19.000000  19.000000  Default  28.0000  28.0000
75        job   7.000000   7.000000  Default  27.0000  27.0000
43   post_hoc  10.000000  10.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
15    improve   0.018541  19.755053  Topic20  -6.5088  -2.4308
16       know   0.018541   7.908065  Topic20  -6.5088  -1.5153
17  knowledge   0.018541   1.942271  Topic20  -6.5088  -0.1113
18       make   0.018541   6.342176  Topic20  -6.5088  -1.2947
20       pain   0.018541   1.449484  Topic20  -6.5088   0.1814

[908 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
339       6  0.523300          able
339      17  0.523300          able
380       3  0.692891        access
161       5  0.707102    accomplish
448       2  0.375859  accomplished
...     ...       ...           ...
248      11  0.036515     would_use
248      14  0.036515     would_use
160       1  0.332911         wrong
160       2  0.332911         wrong
160       6  0.332911         wrong

[811 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 14, 11, 1, 15, 2, 5, 13, 17, 19, 18, 9, 20, 7, 10, 8, 4, 6, 3, 16])

In [157]:
nltk.pos_tag(['much_effort'],tagset='universal')

[('much_effort', 'NOUN')]